In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.fftpack
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# import ipyvolume as ipv
%matplotlib nbagg

A common problem in magnetics is the study of the Heisenberg Hamiltonian in 3-D space, commonly on a regular lattice:

\begin{equation}
\mathcal{H} = -\frac{1}{2}\sum_{ij}S_{i} J_{ij} S_{j} - h \sum_{i} S_i - \sum_{i} S_i \sum_{j} \frac{3(S_j \cdot \hat{r}_{ij})\hat{r}_{ij}}{r_{ij}^5} - \frac{S_j}{r_{ij}^3}
\end{equation}

Commonly the dipolar field calculation is often neglected for large scale calculatiins because the computational complexity of calculating the interaction has $\mathcal{O}(N^2)$ complexity. In some cases, particularly where modelling 2-D systems, it is approximated by including an energy term in the Hamiltonian which has the same form as the Uniaxial Anisotropy, however it has been shown that in 3-D cases this is *not* sufficient, and can give incorrect results when modelling magnetisation dynamics with the Landau-Lifshitz-Gilbert equation. The dipolar interaction is crucial to the stabilisation of many interesting features in the magnetic field, such as domain walls, vortices and skyrmions.

In this notebook, we aim to show how the field can be calculated much more quickly, using a Fast Fourier Transform (FFT) convolution approach.

To start, lets consider a three-dimensional cubic lattice of spins, with $n_x \times n_y \times n_z$ in total.

If we index each axis as (i, j, k) We can give each spin a unique index, given by its position in the lattice, as
\begin{equation}
\text{I} = n_x\times n_y\times k + n_x\times j + i 
\end{equation}

The position of a given spin $I$ is then just given by:
\begin{equation}
\mathbf{r_I} = (i\,a_x, j\,a_y, k\,a_z)
\end{equation}
where $a_x, a_y, a_z$ are the distances between adjacent cubic unit cells along each coordinate axis.

Below, we just plot the magnetisation for a uniformly magnetised sample:

In [2]:
import numpy as np

nx = ny = nz = 5
lenx = 2*nx - 1
leny = 2*ny - 1
lenz = 2*nz - 1
shape = (lenx, leny, lenz)
ax = 1
ay = 1
az = 1

np.random.seed(0)

mx = np.random.uniform(-1, 1, (nz, ny, nz))
my = np.random.uniform(-1, 1, (nz, ny, nz))
mz = np.random.uniform(-1, 1, (nz, ny, nz))
mod = np.sqrt(mx ** 2 + my ** 2 + mz ** 2)
mx /= mod
my /= mod
mz /= mod
Hx = np.zeros_like(mx)
Hy = np.zeros_like(my)
Hz = np.zeros_like(mz)
Hx_fft = np.zeros_like(mx)
Hy_fft = np.zeros_like(my)
Hz_fft = np.zeros_like(mz) 

In [3]:
def H_direct(mx, my, mz, Hx, Hy, Hz):
    Hx[:] = 0
    Hy[:] = 0
    Hz[:] = 0
    for k in range(nz):
        for j in range(ny):
            for i in range(nx):            
                for kp in range(nz):
                    for jp in range(ny):
                        for ip in range(nx):
                            x = (i - ip)*ax
                            y = (j - jp)*ay
                            z = (k - kp)*az
                            dr = np.sqrt(x**2 + y**2 + z**2)
                            # Spins don't interact with themselves, so skip if on same position
                            if (dr == 0):
                                continue
                            r = np.array([x, y, z])
                            S = np.array([mx[kp, jp, ip], my[kp, jp, ip], mz[kp, jp, ip]])
                            H = 1e-7 * (3*(r.dot(S))*r / dr**5 - S/dr**3)
                            Hx[k, j, i] += H[0]
                            Hy[k, j, i] += H[1]
                            Hz[k, j, i] += H[2]

def Nxx_calc(x, y, z):
    x2 = x*x
    y2 = y*y
    z2 = z*z
    r = np.sqrt(x2 + y2 + z2)
    if r == 0:
        return 0.0
    else:
        return (2 * x2 - y2 - z2) / r**5
    
def Nxy_calc(x, y, z):
    r = np.sqrt(x*x + y*y + z*z)
    if r == 0:
        return 0.0
    else:
        return 3*x*y / r**5
    
def ComputeTensor(nx, ny, nz, ax, ay, az):
    lenx = 2*nx - 1
    leny = 2*ny - 1
    lenz = 2*nz - 1
    total = lenx*leny*lenz
    rtotal = lenx*leny*(lenz//2 + 1)
    shape = (lenx, leny, lenz)
    rshape = (lenx, leny, lenz//2 + 1)
    Nxx = np.zeros(shape)
    Nxy = np.zeros_like(Nxx)
    Nxz = np.zeros_like(Nxx)
    Nyy = np.zeros_like(Nxx)
    Nyz = np.zeros_like(Nxx)
    Nzz = np.zeros_like(Nxx)
    for k in range(lenz):
        for i in range(lenx):
            for j in range(leny):
                # Notice that this spans the whole range of -nx*ax to nx*ax
                x = (i - nx + 1) * ax 
                y = (j - ny + 1) * ay
                z = (k - nz + 1) * az
                Nxx[k, j, i] = Nxx_calc(x, y, z)
                Nyy[k, j, i] = Nxx_calc(y, x, z)
                Nzz[k, j, i] = Nxx_calc(z, y, x)
                Nxy[k, j, i] = Nxy_calc(x, y, z)
                Nxz[k, j, i] = Nxy_calc(x, z, y)
                Nyz[k, j, i] = Nxy_calc(y, z, x)
      
    Nxxt = np.fft.rfftn(Nxx)
    Nxyt = np.fft.rfftn(Nxy)
    Nxzt = np.fft.rfftn(Nxz)
    Nyyt = np.fft.rfftn(Nyy)
    Nyzt = np.fft.rfftn(Nyz)
    Nzzt = np.fft.rfftn(Nzz)
    return (Nxx, Nxy, Nxz, Nyy, Nyz, Nzz), (Nxxt, Nxyt, Nxzt, Nyyt, Nyzt, Nzzt)

untransformed, tensor = ComputeTensor(nx, ny, nz, ax, ay, az)

def H_FFT(mx, my, mz, tensor, Hx, Hy, Hz):
    Hx[:] = 0
    Hy[:] = 0
    Hz[:] = 0
    Nxxt, Nxyt, Nxzt, Nyyt, Nyzt, Nzzt = tensor
    nz, ny, nx = mx.shape
    shape = (lenx, leny, lenz)
    mx_padded = np.zeros(shape)
    my_padded = np.zeros(shape)
    mz_padded = np.zeros(shape)
    mx_padded[:nx, :ny, :nz] = mx*1e-7

    my_padded[:nx, :ny, :nz] = my*1e-7
    mz_padded[:nx, :ny, :nz] = mz*1e-7
    Mxt = np.fft.rfftn(mx_padded)
    Myt = np.fft.rfftn(my_padded)
    Mzt = np.fft.rfftn(mz_padded)

    Hxt = (Nxxt*Mxt + Nxyt*Myt + Nxzt*Mzt)
    Hyt = (Nxyt*Mxt + Nyyt*Myt + Nyzt*Mzt)
    Hzt = (Nxzt*Mxt + Nyzt*Myt + Nzzt*Mzt)
    hxc = np.fft.irfftn(Hxt, s=mx_padded.shape)
    hyc = np.fft.irfftn(Hyt, s=mx_padded.shape)
    hzc = np.fft.irfftn(Hzt, s=mx_padded.shape)

    for k in range(nz):
        for j in range(ny):
            for i in range(nx):
                Hx[k, j, i] = hxc[k + nz - 1, j + ny - 1, i + nx - 1]
                Hy[k, j, i] = hyc[k + nz - 1, j + ny - 1, i + nx - 1]
                Hz[k, j, i] = hzc[k + nz - 1, j + ny - 1, i + nx - 1]
    return hxc, hyc, hzc
            


In [4]:
H_direct(mx, my, mz, Hx, Hy, Hz)

In [5]:
hxc, hyc, hzc = H_FFT(mx, my, mz, tensor, Hx_fft, Hy_fft, Hz_fft)

In [6]:
m_init = np.zeros(3*nx*ny*nz)
m_init[::3] = mx.flatten()
m_init[1::3] = my.flatten()
m_init[2::3] = mz.flatten()

In [7]:
import fidimag
import fidimag.atomistic
mesh = fidimag.common.CuboidMesh(nx=nx, ny=ny,nz=nz, dx=ax, dy=ay, dz=az)
sim = fidimag.atomistic.Sim(mesh)
sim.set_m(m_init, normalise=False)
sim.add(fidimag.atomistic.Demag())
demag = sim.interactions[0]
field = demag.compute_field()

In [8]:
print('fidimag Hx')
print(field[::3])
print('direct Hx')
print(Hx.flatten())
print('FFT Hx')
print(Hx_fft.flatten())

fidimag Hx
[ 6.53308572e-08  1.92661797e-07 -9.47893814e-08 -3.09789151e-08
  1.65141599e-07 -8.41042525e-09  2.94130655e-07 -1.30226257e-07
 -1.54693131e-07  3.97684434e-07  9.45515392e-08  4.25471581e-07
 -2.87538910e-08 -2.35206038e-07  1.52605218e-07 -3.44893958e-07
  1.93547006e-07 -2.27057694e-08  3.08505049e-07  2.88079446e-07
  7.26757625e-08  4.19783241e-07  1.60409338e-07 -1.23664835e-07
  7.34071655e-08 -4.35881462e-08 -5.69198656e-08 -3.33477381e-07
  8.06648751e-08  3.38261942e-07 -1.86836749e-09 -2.04293985e-07
 -6.41179183e-08 -2.18836893e-07  2.68295743e-07  5.24611863e-08
  4.15187528e-08  1.88585863e-07  2.72776569e-07  4.28371891e-07
 -1.25644322e-07  5.85113312e-08 -1.37425638e-07  1.69641724e-07
  5.52095050e-08 -1.83174683e-07  1.25866644e-07 -2.71735579e-07
 -1.49131047e-07  1.04323961e-08  7.52541356e-08  1.71581817e-07
 -2.63854470e-07  1.69491607e-08  1.09543499e-07  1.20740555e-07
 -3.86811492e-07  3.96243873e-08 -2.61848786e-08  1.17442445e-07
 -1.88388919e-

In [9]:
print('fidimag Hy')
print(field[1::3])
print('direct Hy')
print(Hy.flatten())
print('FFT Hy')
print(Hy_fft.flatten())

fidimag Hy
[ 1.60298196e-07 -7.63396094e-08 -1.03204848e-08 -7.97268949e-08
 -1.21690053e-08  2.35561865e-08 -2.28469136e-07 -3.85678685e-08
  1.73467925e-07 -5.07936757e-08  2.67160801e-08 -3.57821989e-07
 -3.64178532e-07  3.77945439e-08  8.63439493e-08  4.02415837e-08
 -2.00562012e-07  2.05441306e-07  2.15749523e-07  2.44284751e-07
  3.05539309e-07 -1.36098956e-07  1.93615119e-08  1.21347277e-07
  1.13398272e-07  1.88291275e-07  3.21917388e-07 -4.13464028e-08
 -6.56337819e-10 -5.93982172e-08 -2.13306492e-08 -1.84822525e-09
  2.28178787e-07  4.96439646e-07  2.91264176e-08  1.01048356e-07
 -2.33289155e-07 -9.28610100e-08 -7.35941826e-08 -2.20118301e-07
  7.89646001e-09 -3.38307959e-07  1.03654575e-07  7.75714249e-08
  1.24428697e-07  3.26754864e-07 -6.47576809e-08  5.98292630e-08
  1.09383074e-07 -1.29173363e-07  2.58445723e-08  2.59863781e-08
  8.62200067e-08  5.18972487e-08  2.89855566e-07 -2.23028567e-08
 -1.24991193e-07  2.20771656e-07  2.03691272e-07 -1.53560148e-07
 -2.70613993e-

In [10]:
print('fidimag Hz')
print(field[2::3])
print('direct Hz')
print(Hz.flatten())
print('FFT Hz')
print(Hz_fft.flatten())

fidimag Hz
[ 2.14549546e-07  2.40453893e-07 -5.76768863e-09 -1.49632597e-07
  5.58377274e-08  7.20672248e-08  3.02867401e-07  4.94236593e-08
 -3.14901576e-07 -2.35342441e-07 -7.44750951e-08  1.64408654e-07
 -1.38273976e-07 -2.94972215e-08  2.02530338e-08 -1.59306388e-07
 -2.33061324e-07  1.34934807e-07 -3.32179631e-07  2.57078165e-08
 -3.65048729e-07 -2.12705403e-07 -1.67843439e-08  2.64039410e-07
  1.50928259e-07 -1.07198954e-07  1.56898752e-07 -8.04811635e-08
  9.82094698e-09  1.65362606e-07 -3.68260708e-07 -3.69385627e-07
 -4.00052443e-07  1.00824233e-07 -1.77189701e-07 -1.63117892e-07
 -4.43041568e-07 -8.23824010e-07  6.24885600e-08  2.58253991e-07
  1.23940859e-07 -3.39074435e-07 -1.63675232e-07 -2.69661674e-07
  1.98438140e-09  1.72510239e-07  2.30198048e-08 -2.08974765e-07
 -5.91585903e-09 -2.11340158e-07 -1.55114030e-08  2.77745331e-07
  2.80716485e-07  2.32328706e-07  3.61579779e-07 -3.11513831e-07
  2.77285875e-07  2.05419057e-07  3.82772763e-07  5.38832718e-09
  1.77352686e-